### Goal
Parse *recount2*, gene counts to AnnData format.

Use first `/s/project/gtex-processed/recount/splitGtexData.R` - to split in counts, rowData, colData (the data was available as .RData object)

In [73]:
# code autoreload
%load_ext autoreload
%autoreload 2

import sys
import os

import pandas as pd

from rep import preprocessing as p

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
path = "/s/project/gtex-processed/recount/version2"
counts_file = "recount_gene_counts.csv"
rowdata_file = "recount_rowdata.csv"
coldata_file = "recount_coldata.csv"

In [75]:
# might take longer to create a large object
annobj = p.create_anndata(os.path.join(path,counts_file), sep="\t", samples_anno=os.path.join(path,coldata_file), genes_anno=os.path.join(path,rowdata_file))

In [76]:
sample_description = annobj.var
sample_description.loc[:,'Gender'] = pd.Series(sample_description['bigwig_file']).str.split("_",expand=True)[3]
aux = pd.Series(sample_description['sampid']).str.split("-",expand=True)
sample_description.loc[:,'Individual'] = aux[0].map(str) + "-" + aux[1]
sample_description = sample_description.rename(index=str, columns={"smts": "Parent_Tissue", "smtsd": "Tissue"})
annobj.var = sample_description

In [77]:
annobj.var[:2]

,project,sample,experiment,mapped_read_count,avg_read_length,bigwig_file,sampid,smatsscr,Parent_Tissue,Tissue,Gender,Individual
SRR660824,SRP012682,SRS389722,SRX222703,170790002,152,SRR660824_SRS389722_SRX222703_male_lung.bw,GTEX-QMR6-1926-SM-32PL9,3.0,Lung,Lung,male,GTEX-QMR6
SRR2166176,SRP012682,SRS1036203,SRX1152700,191059974,500,SRR2166176_SRS1036203_SRX1152700_male_brain.ce...,GTEX-T5JC-0011-R11A-SM-5S2RX,NaN,Brain,Brain - Cerebellar Hemisphere,male,GTEX-T5JC


In [78]:
# save AnnData object to HDF5 format
data_path = os.readlink(os.path.join("..","..","data"))
output_file = os.path.join(data_path,"processed","gtex","recount","recount_gtex.h5ad")
p.save(annobj,output_file)

... storing 'symbol' as categorical
... storing 'project' as categorical
... storing 'Parent_Tissue' as categorical
... storing 'Tissue' as categorical
... storing 'Gender' as categorical
... storing 'Individual' as categorical


'/s/project/rep/processed/gtex/recount/recount_gtex.h5ad'